In [ ]:
#from google.colab import drive

from email import utils
import pandas as pd 
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt 
import os 
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras 
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.densenet import DenseNet121
from keras.layers import Dense , GlobalAveragePooling2D , Flatten
from keras.models import Model
from keras import backend as k 
from keras.models import load_model
from tensorflow.keras.optimizers import RMSprop
from Graduation_project import utils

In [ ]:
from tensorflow.python.client import device_lib
print([device.name for device in device_lib.list_local_devices() if device.name != None])
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print('GPUs Available:', len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
os.getcwd()
work_dir = os.getcwd()
image_dir = os.path.join(work_dir,"all_optflow_tvl1_50")
data_df = pd.read_csv(os.path.join(work_dir,"Scripts\label_file.csv"))

In [ ]:
df_train, df_valid, df_test= split_data(data_df)
print(df_train.shape)
print(df_valid.shape)
print(df_test.shape)

In [ ]:
check_overlap(df_train,df_valid, df_test)

In [ ]:
train_generator = train_generator(df_train,image_dir,"image" , "class")
valid_generator  , test_generator = test_and_valid_generator(df_valid, df_test, df_train, image_dir, df_train.columns[0],df_train.columns[1])

In [ ]:
x,y = train_generator.__getitem__(0)
print(len(x)   )

In [ ]:
plt.imshow(x[62].squeeze())

In [ ]:
pos, neg = compute_class_frq(df_train)

In [ ]:
fig = plt.figure()
ax = fig.add_axes([.5,.5,1,1])
ax.bar("static",pos,color = 'r')
ax.bar("dynamic",neg,color = 'b')
ax.legend(labels = ['static','dynamic'])
plt.xlabel("Frames")
plt.ylabel("Percentage")

plt.show()

In [ ]:
from keras.callbacks import ModelCheckpoint

#{epoch:04d} -> for naming unique file after each epoch
checkpoint_path = "saved_model_cp/cp-{epoch:04d}.ckpt"
checkpoint_dir  = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
                checkpoint_path,
                save_weights_only=True,
                verbose=1,
                period= 1, #save each 1 epoch
                )

model = create_model()

In [ ]:
# gpu_slot = 0
# with tf.device('/GPU:0' + str(gpu_slot)):
history = model.fit_generator(
        train_generator, 
        epochs = 5,
        validation_data = valid_generator,
        callbacks = [cp_callback],
        # use_multiprocessing=True,
        # workers = 2
        )
